In [463]:
import pandas as pd

In [464]:
file =open('sample','r', errors='ignore')
file2 =open('replaced','w+')
contents=file.read()
for x in range(len(contents)):

    if contents[x] == ",":
        file2.write("!!@@@")
    else:
        file2.write(str(contents[x]))
file2.close()

In [465]:
names=["Month" ,"Day" ,"Time" , "Machine", "State"]
df= pd.read_csv('replaced', sep=" " , usecols=range(5) ,names=names)

In [466]:
df["State"] = df["State"].replace({':':''}, regex=True)

In [467]:
file3 =open('replaced','r', errors='ignore')
file4 =open('last','w+')
contents2=file3.read()
for x in range(len(contents)):
    if contents2[x] == ":" and contents2[x-1] != "d":
        file4.write("!-!")
    else:
        file4.write(str(contents2[x]))
file4.close()

In [468]:
df2 = pd.read_csv('last', header=None, sep='\n' , encoding='latin-1')
info_only= df2.reset_index()
info_only = info_only[0].str.split(': ', expand=True)


In [469]:
Structure = pd.DataFrame([df["Month"],df["Day"],df["Time"],df["Machine"],df["State"]]) 
Structure=Structure.T

In [470]:
final=pd.concat([Structure ,info_only[1]],axis=1)
final.rename(columns={1:'Info'}, inplace=True)

The next cell changes the value of month to numbers

In [471]:
import datetime
columns= list(final)
for key, value in final["Month"].iteritems():
    month_name = value
    datetime_object = datetime.datetime.strptime(month_name, "%b")
    month_number = datetime_object.month
    final["Month"][key]=month_number


In [472]:
final["Date"]=final.apply(lambda final: datetime.datetime(2020,final["Month"],final['Day']), axis=1)

for key, value in final["Date"].iteritems():
    final["Date"]=datetime.datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S').strftime('%m-%d-%y %H:%M:%S')


In [473]:
for key, value in final["Date"].iteritems():
    out=value.split(' ')
    final["Date"]= out[0]

In [474]:
final=final.drop(columns=["Month","Day"])

In [475]:
cols = final.columns.tolist()
cols = cols[-1:] + cols[:-1]
final=final[cols]

In [476]:
final

,Date,Time,Machine,State,Info
0,07-11-20,12:09:09,dhcp31371,setsid,starting etcd=0
1,07-11-20,12:09:09,dhcp31371,setsid,â etcd.service - Etcd Server
2,07-11-20,12:09:09,dhcp31371,setsid,Loaded
3,07-11-20,12:09:09,dhcp31371,setsid,Active!-! failed (Result!-! start-limit) since...
4,07-11-20,12:09:09,dhcp31371,setsid,Main PID!-! 14281 (code=exited!!@@@ status=1/F...
5,07-11-20,12:09:09,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Fa...
6,07-11-20,12:09:09,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Un...
7,07-11-20,12:09:09,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! et...
8,07-11-20,12:09:09,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! et...
9,07-11-20,12:09:09,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! St...
